In [ ]:
# -*- coding: utf-8 -*-
"""
Colorado Businesses downloader python program
Created on Fri November 29 12:07:50 2021

@author: Dawn Gregg
"""
import json
import requests
import sqlite3
import datetime

# connect to the output database and name it index.sqlite
conn = sqlite3.connect('COBiz.sqlite')
# forces database to retun strings for TEXT attributes
conn.text_factory = str
# get the cursor for the connection
cur = conn.cursor()

# Drop the Tables each time the code is run so you capture changes made to the way data is processed
cur.executescript('''DROP TABLE IF EXISTS CoBiz;
                     DROP TABLE IF EXISTS Address''')

# the Create Table query should include all of the columns you need to process your data
cur.executescript('''CREATE TABLE IF NOT EXISTS CoBiz (entityid TEXT PRIMARY KEY, 
    entityname Text, principaladdress INTEGER, agentprincipaladdress INTEGER, 
    entitystatus TEXT, entitytype TEXT, entityformdate TEXT, year INTEGER);
    CREATE TABLE IF NOT EXISTS Address 
    (addressid INTEGER PRIMARY KEY AUTOINCREMENT, address1 Text, city TEXT, 
    state TEXT, zipcode TEXT, country TEXT)''')

# Initial settings that control the data download
cobizURL =  "https://data.colorado.gov/resource/4ykn-tg5h.json"
paramD = dict()
paramD["principalzipcode"] = 80211                            
paramD["$order"] = "entityformdate DESC"
paramD["$limit"] = 500

# get a document based on base URL plus parameters
document = requests.get(cobizURL, paramD)

# get the JSON text from the URL into a dictionary using the requests library       
if document.status_code == 200 :
    print(document.request.url)
    js = document.json()
else:
    print("Error code=",document.status_code, document.requests.url)
    js = json.loads("{}")

# Output first Record
print("\nFirst Business Reading")
print(js[0])

count = 0
# Loop through entire data set
print("\nAll Entities Downloaded for", paramD["principalzipcode"])
for biz in js:    
    isodate = biz["entityformdate"]
    dt = datetime.datetime.fromisoformat(isodate)
    timestamp = datetime.datetime.timestamp(dt)
          
    # principal address         
    address = biz["principaladdress1"]
    city    = biz["principalcity"]
    zip     = biz["principalzipcode"]
    state   = biz["principalstate"]
    country = biz["principalcountry"]

    # the insert query should include all of the columns you need to add your data
    cur.execute('''INSERT INTO Address (address1, city, state, zipcode, country) 
     VALUES (?,?,?,?,?)''', (address, city, state, zip, country))
     prinaddress_id = cur.lastrowid
         
    try : #Error handling
        # agent principal address         
        address = biz["agentprincipaladdress1"]
        city    = biz["agentprincipalcity"]
        zip     = biz["agentprincipalzipcode"]
        state   = biz["agentprincipalstate"]
        country = biz["agentprincipalcountry"]
         
        # the insert query should include all of the columns you need to add your data
        cur.execute('''INSERT INTO Address (address1, city, state, zipcode, country) 
         VALUES (?,?,?,?,?)''', (address, city, state, zip, country))
         agentaddress_id = cur.lastrowid
    except:
        print("\nError: No Agent Principle for", biz["entityname"])
        # the insert query creates an empty address
        cur.execute('''INSERT OR IGNORE INTO Address (addressid) VALUES (?)''', (0,))
         agentaddress_id = 0
     
    # the insert query should include all of the columns you need to add your data
    cur.execute('''INSERT INTO CoBiz (entityid, entityname, principaladdress, agentprincipaladdress, 
     entitystatus, entitytype, entityformdate, year)  VALUES (?,?,?,?,?,?,?,?)''',
     (biz["entityid"], biz["entityname"], prinaddress_id, agentaddress_id,
     biz["entitystatus"], biz["entitytype"], timestamp, dt.year))

    #increment count
    count = count+1    
         
    # commit the insert statemnt(s) for every 100 records so do not slow program down       
    if count%100==0: 
        print("\nDate Business Started:", isodate)
        print("Business Name:", biz["entityname"])
        conn.commit()

# Remember to commit after the end of the loop to make sure ALL data is saved!!    
print(count,"Colorado Businesses Processed")
conn.commit()